In [ ]:
import geopandas as gpd
import pandas as pd
import json
import plotly.express as px
import zipfile
from urllib import request
import numpy as np
import datetime as DT

In [ ]:
response = request.urlopen('https://services1.arcgis.com/Nifc7wlHaBPig3Q3/arcgis/rest/services/COVID_PA_Counties/FeatureServer/0/query?f=json&where=1%3D1&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&resultOffset=0&resultRecordCount=68&resultType=standard&cacheHint=true')
dat = response.read()
data = json.loads(dat)
df = pd.json_normalize(data['features'])
df.columns=df.columns.str.replace("attributes.",'',regex=False).str.lower()
df=df[['county','cases_per_100000']][df.cases_per_100000.notna()]
df['county']=df.county.str.title()
map1=gpd.read_file("https://data.pa.gov/api/geospatial/2axz-wupj?method=export&format=GeoJSON")
map1['county']=map1.county_nam.str.title()
map1=map1[['county','geometry']]
map1.to_file("map1.geojson",driver='GeoJSON')
with open ("map1.geojson",'r') as infile:
    counties = json.load(infile)
fig = px.choropleth_mapbox(df,geojson=counties,
                          locations='county',featureidkey='properties.county',
                          color='cases_per_100000',
                          hover_name='county',
                          range_color=(df.cases_per_100000.describe()[1]-df.cases_per_100000.describe()[2],df.cases_per_100000.describe()[1]+df.cases_per_100000.describe()[2]),
                          mapbox_style='carto-positron',
                          color_continuous_scale='viridis',
                          zoom=6.0,center = {"lat": 40.9998307, "lon": -77.6162591},
                          opacity=0.5,title='Total Cases Per 100,000 people by county')
fig.show()
#df.to_csv("county_covid_19_"+str(today.month)+"_"+str(today.day)+".csv",index=None)

In [ ]:
url='https://services1.arcgis.com/Nifc7wlHaBPig3Q3/arcgis/rest/services/Covid_Cases_County/FeatureServer/0/query?f=json&where=1%3D1&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&resultOffset=0&resultRecordCount=50000&resultType=standard&cacheHint=true'
response = request.urlopen(url)
dat = response.read()
data = json.loads(dat)
df = pd.json_normalize(data['features'])
df.columns=df.columns.str.replace("attributes.",'',regex=False).str.lower()
df['date']=pd.to_datetime(df['date'],unit="ms",origin='unix')
df=df[df.date==df.date.max()]
df['county']=df.county.str.title()
df=df[['county','cases_avg_new_rate']][df.cases_avg_new_rate.notna()]
map1=gpd.read_file("https://data.pa.gov/api/geospatial/2axz-wupj?method=export&format=GeoJSON")
map1['county']=map1.county_nam.str.title()
map1=map1[['county','geometry']]
map1.to_file("map1.geojson",driver='GeoJSON')
with open ("map1.geojson",'r') as infile:
    counties = json.load(infile)
fig = px.choropleth_mapbox(df,geojson=counties,
                          locations='county',featureidkey='properties.county',
                          color='cases_avg_new_rate',
                          hover_name='county',
                          range_color=(df.cases_avg_new_rate.describe()[1]-df.cases_avg_new_rate.describe()[2],df.cases_avg_new_rate.describe()[1]+df.cases_avg_new_rate.describe()[2]),
                          mapbox_style='carto-positron',
                          color_continuous_scale='viridis',
                          zoom=6.0,center = {"lat": 40.9998307, "lon": -77.6162591},
                          opacity=0.5,title="7-day average new cases per 100,000 people by county")
fig.show()